In [ ]:
# Import Libraries 

!pip3 install matplotlib
!pip3 install scikit-learn

import pandas as pd
import plotly.express as pe
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# Import Data from Google Sheets 

google_sheets_url = "https://docs.google.com/spreadsheets/d/1E0lyCSxlC0ajNtzjpWo17TX5DEeEjd33E-j6c7fOBcg/export?format=csv&gid=199229195"

df_full_country_list= pd.read_csv(google_sheets_url)

df_full_country_list

In [ ]:
df_full_country_list

In [123]:
# 1. Filter the DataFrame by Year

# Define the filtering parameters
START_YEAR = 2000
END_YEAR = 2023
COUNTRIES_TO_DROP = ['China', "Cote d'Ivoire", "Ghana"]

## 1. Filter by Year AND Drop Countries
df_final_working = df_full_country_list[
    # Filter for years between 2000 and 2023
    (df_full_country_list['Year'] >= START_YEAR) & 
    (df_full_country_list['Year'] <= END_YEAR) &
    
    # Drop the specified countries using the negation operator (~)
    (~df_full_country_list['Country Name'].isin(COUNTRIES_TO_DROP))
].copy()

In [122]:
# ---------                                                        --------
#              Create economic success indicator 
# ---------                                                        --------

# 1. Start with pivoting dataframe to ensure GDP, Unemployment and Inflation each have an individual column  

df_esi = df_final_working.pivot_table(
    index=['Country Name', 'Year'],
    columns='Indicator Name',
    values='Value'
).reset_index()

# 2. Define a list to include three indicators used: GDP, Unemplyoment and Inflation

Econ_indicators = [
    'GDP per capita',
    'Unemployment levels (%)',
    'Inflation (CPI, %))'
]



# 3. Concatenate DF with indicator list and rename columns 

df_esi = df_econ[['Country Name', 'Year'] + ESI_indicators].copy()

df_esi.rename(columns={
    'GDP per capita': 'GDP',
    'Unemployment levels (%)': 'UNEMP',
    'Inflation (CPI, %))': 'INFLATION'
}, inplace=True)



In [124]:
# 4. Recode directionality and standardize to prepare data for principal component analysis 

X_econ = df_esi[['GDP', 'UNEMP', 'INFLATION']].copy()

# 4.1 Invert the unemployment and inflation indicators by multiplying by -1.
X_econ_recoded = X_econ.copy()
X_econ_recoded['UNEMP'] = X_econ_recoded['UNEMP'] * -1
X_econ_recoded['INFLATION'] = X_econ_recoded['INFLATION'] * -1

# 4.2 Standardize (Z-score) the recoded data.
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_econ_recoded)


In [ ]:
# 5. Apply principal component analysis

# 5.1 Initialize PCA
pca_econ = PCA()
pca_econ.fit(X_standardized)

# 5.2 Transform the standardized data to get the scores for each country-year
PC_scores = pca_esi.transform(X_standardized)

# 5.4 The first Principal Component (PC1) is the Economic Success Indicator
df_econ_working['Economic Success (PCA)'] = PC_scores[:, 0]


# 5.5 REVIEW PCA OUTPUT (Crucial for Documentation) ---

print("--- DataFrame Head with Economic Success (PCA) Score ---")
print(df_econ_working[['Country Name', 'Year', 'Economic Success (PCA)']].head())

print("\n--- PCA Loadings (Weights for ESI) ---")
# These loadings are the statistically optimal weights for your ESI
loadings = pd.Series(pca_esi.components_[0], index=X_econ_recoded.columns)
print(loadings)

print("\n--- Explained Variance Ratio ---")
# This tells you the percentage of total variance the ESI captures.
# The first number in the array is the most important: the percentage explained by PC1.
print(pca_esi.explained_variance_ratio_)

In [128]:
df_econ_working

Indicator Name,Country Name,Year,GDP,UNEMP,INFLATION,Economic Success (PCA)
0,Chile,2000,5099.922,9.708,57.794,-1.158580
1,Chile,2001,4608.057,9.867,59.317,-1.170196
2,Chile,2002,4479.771,9.800,60.993,-1.152127
3,Chile,2003,4824.870,9.533,61.647,-1.114838
4,Chile,2004,6183.906,10.017,63.144,-1.097073
...,...,...,...,...,...,...
211,United States,2019,65561.320,3.675,258.353,3.090645
212,United States,2020,64518.063,8.100,262.389,2.758991
213,United States,2021,71365.337,5.358,281.782,3.365810
214,United States,2022,77944.095,3.642,299.845,3.872372


In [ ]:
fig = pe.line(
    df_econ_working,
    x='Year',                 
    y='Economic Success (PCA)', 
    color='Country Name',     
    title='ESI Trend'
)

fig.show()

In [ ]:
## Create Well-Being Indicator



